In [17]:
import time 
import logging
import requests
import re
import sys
from bs4 import BeautifulSoup
from queue import Queue
from urllib import parse, request
from urllib.parse import urlparse
import heapq
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.by import By

In [18]:
site = "https://ocw.mit.edu/search/?d=Electrical%20Engineering%20and%20Computer%20Science"
page = requests.get(site)
soup = BeautifulSoup(page.content, "html.parser")

In [19]:
# Create a webdriver instance
driver = webdriver.Chrome()

# Get the main page with the links to the courses
driver.get('https://ocw.mit.edu/search/?d=Electrical%20Engineering%20and%20Computer%20Science&q=computer%20science')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
# Scroll to the bottom of the page to load more content
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for new content to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [21]:
# Find all links to courses
cards = driver.find_elements(By.CLASS_NAME, 'lr-row.course-title')

# Extract links from each card
links = []
for card in cards:
    link_element = card.find_element(By.TAG_NAME, 'a')
    link = link_element.get_attribute("href")
    links.append(link)

#print(links)

In [22]:
# Visit each link and extract the title and description

titles = []
descriptions = []

for link in links:
    driver.get(link)
    time.sleep(2)  # Add a delay to allow the page to load
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract title
    title_element = soup.find('a', class_='text-capitalize m-0 text-white')
    if title_element:
        titles.append(title_element.text.strip())
    else:
        titles.append('Title not found')
    
    # Extract description
    description_element = soup.find('div', id='expanded-description')
    if description_element:
        descriptions.append(description_element.text.strip())
    else:
        description_element = soup.find('div', id='full-description')
        if description_element:
            descriptions.append(description_element.text.strip()) 
        else:
            descriptions.append('Description not found') 
    
    #print(f"Title: {title}")
    #print(f"Description: {description}")
    #print()

# Quit the webdriver
driver.quit()





In [23]:
course_df = pd.DataFrame(
    {'Title': titles,
     'Descriptions': descriptions})

In [25]:
course_df.head()

,Title,Descriptions
0,Mathematics for Computer Science,This course covers elementary discrete mathema...
1,Mathematics for Computer Science,This is an introductory course in Discrete Mat...
2,Mathematics for Computer Science,This subject offers an interactive introductio...
3,Computational Cognitive Science,This course is an introduction to computationa...
4,Introduction to Computer Science and Programming,6.00 Intro to CS and Programming has been reti...


In [26]:
course_df.to_csv('course_df.csv')

In [16]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained sentence transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example course description
course_description = """
6.776 covers circuit level design issues of high speed communication systems, with primary focus being placed on wireless and broadband data link applications. Specific circuit topics include transmission lines, high speed and low noise amplifiers, VCO’s, mixers, power amps, high speed digital circuits, and frequency synthesizers. In addition to learning analysis skills for the above items, students will gain a significant amount of experience in simulating RF circuits in SPICE and also building RF circuits within a lab project.
"""

# User input
user_goals = input("What are your goals for an online CS course? ")

# Encode the user's goals and the course description
user_goals_embedding = model.encode(user_goals, convert_to_tensor=True)
course_description_embedding = model.encode(course_description, convert_to_tensor=True)

# Calculate the similarity between the embeddings
cosine_score = util.pytorch_cos_sim(user_goals_embedding, course_description_embedding).item()

print(f"Similarity Score: {cosine_score}")


What are your goals for an online CS course?  I want to learn how graph theory and discrete math as well as sets and counting techniques fit into computer science.


Similarity Score: 0.14673715829849243
